In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog, local_binary_pattern

In [2]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),  
])

valid_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
])


from torchvision import datasets

train_dataset = datasets.ImageFolder(
    root="train",
    transform=train_transform
)

valid_dataset = datasets.ImageFolder(
    root="val",
    transform=valid_transform
)


In [3]:
def dataset_to_numpy(dataset):
    images = []
    labels = []

    for img, label in dataset:
        img_np = np.array(img, dtype=np.float32) / 255.0  # normalize to [0,1]
        images.append(img_np)
        labels.append(label)

    return np.array(images), np.array(labels)


In [4]:
train_images, train_labels = dataset_to_numpy(train_dataset)
test_images, test_labels = dataset_to_numpy(valid_dataset)

print(train_images.shape)  # (N, H, W)
print(train_labels.shape)


(16584, 48, 48)
(16584,)


In [5]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, feature_type='hog'):
        self.feature_type = feature_type

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.feature_type == 'hog':
            return np.array([hog(img, orientations=8, pixels_per_cell=(4, 4),
                                 cells_per_block=(1, 1), visualize=False) for img in X])
        elif self.feature_type == 'lbp':
            return np.array([local_binary_pattern(img, P=8, R=1, method='uniform').flatten()
                             for img in X])
        elif self.feature_type == 'histogram':
            return np.array([np.histogram(img, bins=32, range=(0, 1))[0] for img in X])
        else:
            raise ValueError("Invalid feature type. Choose 'hog', 'lbp', or 'histogram'.")

In [6]:
# Extract once
extractor = FeatureExtractor('hog')
X_train_feat = extractor.fit_transform(train_images)
X_test_feat = extractor.transform(test_images)

# Cache
np.save("X_train_hog.npy", X_train_feat)
np.save("X_test_hog.npy", X_test_feat)


In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_feat)
X_test_scaled = scaler.transform(X_test_feat)

pca = PCA(
    n_components=0.95,
    svd_solver='auto',
    random_state=42
)


X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("Final feature shape:", X_train_pca.shape)


Final feature shape: (16584, 913)


In [8]:
from sklearn.svm import LinearSVC

classifiers = {
    'Linear SVM': LinearSVC(C=1.0,loss='squared_hinge',max_iter=2000,tol=1e-3,random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5, weights='distance'),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=42)
}

In [9]:
results = {}

for name, clf in classifiers.items():
    print(f"\nTraining {name}...")
    clf.fit(X_train_pca, train_labels)

    predictions = clf.predict(X_test_pca)

    print(f"{name} Results:")
    print(classification_report(test_labels, predictions))

    results[name] = classification_report(
        test_labels, predictions, output_dict=True
    )



Training Linear SVM...
Linear SVM Results:
              precision    recall  f1-score   support

           0       0.56      0.54      0.55       403
           1       0.70      0.86      0.77      1782
           2       0.36      0.06      0.10       176
           3       0.45      0.30      0.36       681
           4       0.78      0.79      0.78       510

    accuracy                           0.66      3552
   macro avg       0.57      0.51      0.51      3552
weighted avg       0.63      0.66      0.64      3552


Training KNN...
KNN Results:
              precision    recall  f1-score   support

           0       0.57      0.44      0.50       403
           1       0.66      0.84      0.74      1782
           2       0.15      0.06      0.09       176
           3       0.38      0.32      0.35       681
           4       0.93      0.58      0.72       510

    accuracy                           0.62      3552
   macro avg       0.54      0.45      0.48      3552
wei

In [10]:
# Compare model performance
print("\nModel Comparison:")
for model, metrics in results.items():
    print(f"{model}:")
    print(f"Accuracy: {metrics['accuracy']:.3f}")
    print(f"Macro Avg F1: {metrics['macro avg']['f1-score']:.3f}\n")


Model Comparison:
Linear SVM:
Accuracy: 0.664
Macro Avg F1: 0.512

KNN:
Accuracy: 0.620
Macro Avg F1: 0.478

Decision Tree:
Accuracy: 0.542
Macro Avg F1: 0.377

